

  **Abdullah Farooq - 2020023** |
  **Ammar Ahmad - 2020071** |
  **Bilal Malik - 2020102**


## Grammar

● program -> stmt_list

● stmt_list -> stmt stmt_list | ε

● stmt -> assignment_stmt | if_stmt | print_stmt

● assignment_stmt -> VARIABLE = expr ;

● if_stmt -> if ( expr ) { stmt_list } else { stmt_list } | if ( expr ) { stmt_list }

● print_stmt -> print ( expr ) ;

● expr -> term expr'

● expr' -> + term expr' | - term expr' | ε

● term -> factor term'

● term' -> * factor term' | ε

● factor -> NUMBER | VARIABLE | (expr)

● NUMBER -> [0-9]+

● VARIABLE -> [a-zA-Z]+


## Code

In [16]:
import re

class Parser:
    def __init__(self, input_string):
        # Tokenize the input string and initialize parser variables
        self.input_tokens = self.tokenize(input_string)
        self.current_token = None
        self.pos = -1
        self.advance()

    def tokenize(self, input_string):
        # Tokenize the input string using regular expressions
        # Tokens include identifiers, numbers, operators, and special characters
        return re.findall(r'//.*?$|/\*.*?\*/|\b\w+\b|[^\w\s]', input_string, re.MULTILINE|re.DOTALL)

    def advance(self):
        # Move to the next token in the input
        self.pos += 1
        if self.pos < len(self.input_tokens):
            self.current_token = self.input_tokens[self.pos]
        else:
            self.current_token = None

    def error(self):
        # Raise a syntax error
        raise Exception("Syntax Error")

    def expr(self):
        # Parse an expression following the grammar rules
        result = self.term()
        while self.current_token in ('+', '-'):
            op = self.current_token
            self.advance()
            result = (op, result, self.term())
        return result

    def term(self):
        # Parse a term following the grammar rules
        result = self.factor()
        while self.current_token in ('*', '/'):
            op = self.current_token
            self.advance()
            result = (op, result, self.factor())
        return result

    def factor(self):
        # Parse a factor following the grammar rules
        if self.current_token.isdigit():
            return self.number()
        elif self.current_token.isalpha():
            return self.variable()
        elif self.current_token == '(':
            self.advance()
            result = self.expr()
            if self.current_token != ')':
                self.error()
            self.advance()
            return result
        else:
            self.error()

    def number(self):
        # Parse a number token
        result = ''
        while self.current_token is not None and self.current_token.isdigit():
            result += self.current_token
            self.advance()
        return int(result)

    def variable(self):
        # Parse a variable token
        result = ''
        while self.current_token is not None and self.current_token.isalpha():
            result += self.current_token
            self.advance()
        return result

    def assignment_stmt(self):
        # Parse an assignment statement
        variable = self.variable()
        if self.current_token != '=':
            self.error()
        self.advance()
        expr_result = self.expr()
        if self.current_token != ';':
            self.error()
        self.advance()
        return ('=', variable, expr_result)

    def if_stmt(self):
        # Parse an if statement
        if self.current_token != 'if':
            self.error()
        self.advance()
        if self.current_token != '(':
            self.error()
        self.advance()
        condition = self.expr()
        if self.current_token not in ('<', '>', '<=', '>=', '=='):
            self.error()
        op = self.current_token
        self.advance()
        second_condition = self.expr()
        if self.current_token != ')':
            self.error()
        self.advance()
        if_block = self.stmt_list()
        else_block = []
        if self.current_token == 'else':
            self.advance()
            else_block = self.stmt_list()
        return ('if', condition, op, second_condition, if_block, else_block)

    def print_stmt(self):
        # Parse a print statement
        if self.current_token != 'print':
            self.error()
        self.advance()
        if self.current_token != '(':
            self.error()
        self.advance()
        expr_result = self.expr()
        if self.current_token != ')':
            self.error()
        self.advance()
        if self.current_token != ';':
            self.error()
        self.advance()
        return ('print', expr_result)

    def stmt(self):
        # Determine the type of statement and call the corresponding parsing method
        if self.current_token == 'if':
            return self.if_stmt()
        elif self.current_token == 'print':
            return self.print_stmt()
        else:
            return self.assignment_stmt()

    def stmt_list(self):
        # Parse a list of statements
        statements = []
        while self.current_token is not None and self.current_token != '}':
            statement = self.stmt()
            statements.append(statement)
        return statements

    def parse(self):
        # Entry point for parsing, starts with stmt_list
        return self.stmt_list()

def test_parser(expression):
    # Test the parser with an input expression
    parser = Parser(expression)
    try:
        result = parser.parse()
        statement_list = parser.tokenize(expression)
        print("Tokens List:", statement_list)
        print("Expression:", expression)
        print("Parsed Result:", result)
    except Exception as e:
        print("Expression:", expression)
        print("Error:", e)
    print()

# Test the parser with various input expressions
test_parser("a = 5;")
test_parser("x = 5; print(x / 5);")
test_parser("print(4 + 7 + 5);")
test_parser("if (x > 2) print(x);")


test_parser("a \ 5") # Invalid syntax
test_parser("3 * [ b - 2]") # Invalid syntax
test_parser("x / {y + 7}")  # Invalid syntax
test_parser("a = 5") # Invalid syntax


test_parser("if (x > 2) print(x); else print(z); ") # Valid Syntax but not parsing accurately

Tokens List: ['a', '=', '5', ';']
Expression: a = 5;
Parsed Result: [('=', 'a', 5)]

Tokens List: ['x', '=', '5', ';', 'print', '(', 'x', '/', '5', ')', ';']
Expression: x = 5; print(x / 5);
Parsed Result: [('=', 'x', 5), ('print', ('/', 'x', 5))]

Tokens List: ['print', '(', '4', '+', '7', '+', '5', ')', ';']
Expression: print(4 + 7 + 5);
Parsed Result: [('print', ('+', ('+', 4, 7), 5))]

Tokens List: ['if', '(', 'x', '>', '2', ')', 'print', '(', 'x', ')', ';']
Expression: if (x > 2) print(x);
Parsed Result: [('if', 'x', '>', 2, [('print', 'x')], [])]

Expression: a \ 5
Error: Syntax Error

Expression: 3 * [ b - 2]
Error: Syntax Error

Expression: x / {y + 7}
Error: Syntax Error

Expression: a = 5
Error: Syntax Error

Expression: if (x > 2) print(x); else {print(z);} 
Error: Syntax Error



# Intermediate code

In [12]:
import re

class Parser:
    def __init__(self, input_string):
        self.input_tokens = self.tokenize(input_string)
        self.current_token = None
        self.pos = -1
        self.advance()

    def tokenize(self, input_string):
        return re.findall(r'//.*?$|/\*.*?\*/|\b\w+\b|[^\w\s]', input_string, re.MULTILINE|re.DOTALL)

    def advance(self):
        self.pos += 1
        if self.pos < len(self.input_tokens):
            self.current_token = self.input_tokens[self.pos]
        else:
            self.current_token = None

    def error(self):
        raise Exception("Syntax Error")

    def expr(self):
        result = self.term()
        while self.current_token in ('+', '-'):
            op = self.current_token
            self.advance()
            result = (op, result, self.term())
        return result

    def term(self):
        result = self.factor()
        while self.current_token in ('*', '/'):
            op = self.current_token
            self.advance()
            result = (op, result, self.factor())
        return result

    def factor(self):
        if self.current_token.isdigit():
            return self.number()
        elif self.current_token.isalpha():
            return self.variable()
        elif self.current_token == '(':
            self.advance()
            result = self.expr()
            if self.current_token != ')':
                self.error()
            self.advance()
            return result
        else:
            self.error()

    def number(self):
        result = ''
        while self.current_token is not None and self.current_token.isdigit():
            result += self.current_token
            self.advance()
        return ('NUMBER', int(result))

    def variable(self):
        result = ''
        while self.current_token is not None and self.current_token.isalpha():
            result += self.current_token
            self.advance()
        return ('VARIABLE', result)

    def assignment_stmt(self):
        variable = self.variable()
        if self.current_token != '=':
            self.error()
        self.advance()
        expr_result = self.expr()
        if self.current_token != ';':
            self.error()
        self.advance()
        return ('ASSIGN', variable, expr_result)

    def if_stmt(self):
        if self.current_token != 'if':
            self.error()
        self.advance()
        if self.current_token != '(':
            self.error()
        self.advance()
        condition = self.expr()
        if self.current_token not in ('<', '>', '<=', '>=', '=='):
            self.error()
        op = self.current_token
        self.advance()
        second_condition = self.expr()
        if self.current_token != ')':
            self.error()
        self.advance()
        if self.current_token != '{':
            self.error()
        self.advance()
        if_block = self.stmt_list()
        if self.current_token != '}':
            self.error()
        self.advance()
        else_block = []
        if self.current_token == 'else':
            self.advance()
            if self.current_token != '{':
                self.error()
            self.advance()
            else_block = self.stmt_list()
            if self.current_token != '}':
                self.error()
            self.advance()
        return ('IF', condition, op, second_condition, if_block, else_block)

    def print_stmt(self):
        if self.current_token != 'print':
            self.error()
        self.advance()
        if self.current_token != '(':
            self.error()
        self.advance()
        expr_result = self.expr()
        if self.current_token != ')':
            self.error()
        self.advance()
        if self.current_token != ';':
            self.error()
        self.advance()
        return ('PRINT', expr_result)

    def stmt(self):
        if self.current_token == 'if':
            return self.if_stmt()
        elif self.current_token == 'print':
            return self.print_stmt()
        else:
            return self.assignment_stmt()

    def stmt_list(self):
        statements = []
        while self.current_token is not None and self.current_token != '}':
            statement = self.stmt()
            statements.append(statement)
        return statements

    def parse(self):
        return self.stmt_list()

class IntermediateCodeGenerator:
    def __init__(self):
        self.temp_count = 0
        self.code = []

    def new_temp(self):
        temp = f"t{self.temp_count}"
        self.temp_count += 1
        return temp

    def generate(self, node):
        if isinstance(node, tuple):
            if node[0] == 'NUMBER':
                return node[1]
            elif node[0] == 'VARIABLE':
                return node[1]
            elif node[0] == 'ASSIGN':
                var = node[1][1]
                expr = self.generate(node[2])
                self.code.append(f"{var} = {expr}")
                return var
            elif node[0] in ('+', '-', '*', '/'):
                left = self.generate(node[1])
                right = self.generate(node[2])
                temp = self.new_temp()
                self.code.append(f"{temp} = {left} {node[0]} {right}")
                return temp
            elif node[0] == 'IF':
                cond1 = self.generate(node[1])
                cond_op = node[2]
                cond2 = self.generate(node[3])
                self.code.append(f"if {cond1} {cond_op} {cond2} goto L{self.temp_count}")
                if_block = self.generate_block(node[4])
                self.code.append(f"goto L{self.temp_count + 1}")
                self.code.append(f"L{self.temp_count}:")
                else_block = self.generate_block(node[5])
                self.code.append(f"L{self.temp_count + 1}:")
                self.temp_count += 2
            elif node[0] == 'PRINT':
                expr = self.generate(node[1])
                self.code.append(f"print {expr}")
                return expr
        else:
            self.error()

    def generate_block(self, block):
        for stmt in block:
            self.generate(stmt)

    def error(self):
        raise Exception("Intermediate Code Generation Error")

    def generate_code(self, ast):
        self.generate_block(ast)
        return self.code

def test_parser_and_codegen(expression):
    parser = Parser(expression)
    try:
        ast = parser.parse()
        print("Expression:", expression)
        print("Tokens: ", parser.tokenize(expression))
        print("AST:", ast)
        codegen = IntermediateCodeGenerator()
        code = codegen.generate_code(ast)
        print("Intermediate Code:")
        for line in code:
            print(line)
    except Exception as e:
        print("Expression:", expression)
        print("Error:", e)
    print()

# Test example
test_parser_and_codegen("a = 5;")
test_parser_and_codegen("x = 5; print(x / 5);")
test_parser_and_codegen("print(4 + 7 + 5);")
test_parser_and_codegen("if (x > 2) {print(x);}")

Expression: a = 5;
Tokens:  ['a', '=', '5', ';']
AST: [('ASSIGN', ('VARIABLE', 'a'), ('NUMBER', 5))]
Intermediate Code:
a = 5

Expression: x = 5; print(x / 5);
Tokens:  ['x', '=', '5', ';', 'print', '(', 'x', '/', '5', ')', ';']
AST: [('ASSIGN', ('VARIABLE', 'x'), ('NUMBER', 5)), ('PRINT', ('/', ('VARIABLE', 'x'), ('NUMBER', 5)))]
Intermediate Code:
x = 5
t0 = x / 5
print t0

Expression: print(4 + 7 + 5);
Tokens:  ['print', '(', '4', '+', '7', '+', '5', ')', ';']
AST: [('PRINT', ('+', ('+', ('NUMBER', 4), ('NUMBER', 7)), ('NUMBER', 5)))]
Intermediate Code:
t0 = 4 + 7
t1 = t0 + 5
print t1

Expression: if (x > 2) {print(x);}
Tokens:  ['if', '(', 'x', '>', '2', ')', '{', 'print', '(', 'x', ')', ';', '}']
AST: [('IF', ('VARIABLE', 'x'), '>', ('NUMBER', 2), [('PRINT', ('VARIABLE', 'x'))], [])]
Intermediate Code:
if x > 2 goto L0
print x
goto L1
L0:
L1:

